# Policy Learning Only Workflow

In this workflow, CFRL takes in an offline trajectory and then preprocesses the offline trajectory using `SequentialPreprocessor`. After that, the preprocessed trajectory is passed into `FQI` to train a counterfactually fair policy, which is the final output of the workflow. This workflow is appropriate if the user wants to train a policy using CFRL. The trained policy can be further evaluated on its value and counterfactual fairness, which is discussed in detail in the "Assessing Policies Using Real Data" workflow later in this section.

We begin by importing the liberaries needed for this demonstration.

In [1]:
# Need this temporarily to import CFRL before it is officially published to PyPI
import sys
sys.path.append("E:/learning/university/MiSIL/CFRL Python Package/CFRL")

In [2]:
import pandas as pd
import numpy as np
#import pytorch as torch
from sklearn.model_selection import train_test_split
from CFRL.reader import read_trajectory_from_dataframe, convert_trajectory_to_dataframe
from CFRL.preprocessor import SequentialPreprocessor
from CFRL.agents import FQI
np.random.seed(1) # ensure reproducibility
#torch.manual_seed(1)

## Data Loading

In this demonstration, we use an offline trajectory generated from a `SyntheticEnvironment` using some pre-specified transition rules. Although it is actually synthesized, we treat it as if it is from some unknown environment for pedagogical convenience in this demonstration.

The trajectory contains 500 individuals (i.e. $N=500$) and 10 transitions (i.e. $T=10$). The actions are binary (0 or 1) and were sampled using a random policy that selects 0 or 1 randomly with equal probability. It is stored in a tabular format in a `.csv` file. The sensitive attribute variable is bivariate, stored in columns `z1` and `z2`. The legit values of the sensitive attribute are $[0, 0]$, $[1, 0]$, $[0, 1]$, and $[1, 1]$. The state variable is tri-variate, stored in columns `state1`, `state2`, and `state3`. The actions are stored in the column `action` and rewards in the column `reward`. The tabular data also includes an extra inrrelevant column `timestamp`. 

We can load and view the tabular data.

In [3]:
trajectory = pd.read_csv('../data/sample_data_large_multi.csv')
trajectory

,Unnamed: 0,ID,timestamp,z1,z2,action,reward,state1,state2,state3
0,0,1.0,1.0,0.0,0.0,NaN,NaN,2.124345,-0.111756,-0.028172
1,1,1.0,2.0,0.0,0.0,1.0,3.380339,-0.071876,0.545250,-0.020279
2,2,1.0,3.0,0.0,0.0,0.0,1.849111,-1.084077,-1.696634,-1.179136
3,3,1.0,4.0,0.0,0.0,0.0,-4.421291,-2.317520,-1.787875,-2.148363
4,4,1.0,5.0,0.0,0.0,1.0,-5.142691,-2.936506,-3.603797,-3.590126
...,...,...,...,...,...,...,...,...,...,...
5495,5495,500.0,7.0,0.0,0.0,0.0,-12.563265,-4.024293,-6.587401,-3.859436
5496,5496,500.0,8.0,0.0,0.0,0.0,-14.073520,-5.952644,-5.854450,-4.218220
5497,5497,500.0,9.0,0.0,0.0,0.0,-16.691358,-5.687570,-6.008377,-5.618730
5498,5498,500.0,10.0,0.0,0.0,0.0,-18.394408,-7.551435,-6.816310,-6.740886


We now read the trajectory from the tabular format into Trajectory Arrays.

In [4]:
zs, states, actions, rewards, ids = read_trajectory_from_dataframe(
                                                data=trajectory, 
                                                z_labels=['z1', 'z2'], 
                                                state_labels=['state1', 'state2', 'state3'], 
                                                action_label='action', 
                                                reward_label='reward', 
                                                id_label='ID', 
                                                T=10
                                                )

## Preprocessor Training

Before preprocessing the trajectory, we need to first train a preprocessor. To mitigate overfitting, we use a random subset of 250 individuals in the trajectory to train the preprocessor. The remaining 250 individuals will be actually preprocessed. We now form these two sets.

In [5]:
(
    zs_train, zs_prepro, 
    states_train, states_prepro, 
    actions_train, actions_prepro, 
    rewards_train, rewards_prepro, 
    ids_train, ids_prepro
) = train_test_split(zs, states, actions, rewards, ids, test_size=0.5)

We now use the training set to train a `SequentialPreprocessor`.

In [6]:
sp = SequentialPreprocessor(z_space=[[0, 0], [0, 1], [1, 0], [1, 1]], 
                            num_actions=2, 
                            cross_folds=1, 
                            mode='single', 
                            reg_model='nn')
sp.train_preprocessor(zs=zs_train, xs=states_train, actions=actions_train, rewards=rewards_train)

100%|██████████| 1000/1000 [00:39<00:00, 25.31it/s]


(array([[[  0.29730609,   0.64565182,  -1.68102797, ...,   2.47447229,
            2.52162402,   0.375634  ],
         [ -1.35034815,  -2.25309663,  -1.63382889, ...,   2.77911791,
            1.54900343,   1.38914679],
         [ -3.42806811,  -4.52740321,  -3.80843044, ...,   1.76331143,
            0.3825182 ,   1.79272496],
         ...,
         [ -6.80947176,  -7.44296927,  -6.91436753, ...,   5.74308356,
            3.70575099,   4.58800617],
         [ -8.76876788,  -7.18474406,  -8.88386579, ...,   3.65733714,
            4.32011253,   4.03171258],
         [ -9.04944974,  -9.08450819,  -9.40940367, ...,   4.41984767,
            4.20582365,   4.18209709]],
 
        [[  0.30006061,   0.50738898,   0.77566408, ...,   2.47722681,
            2.38336118,   2.83232604],
         [ -1.27571934,  -1.32202527,  -2.06041935, ...,   3.00406546,
            2.32318667,   1.76551351],
         [ -3.04574095,  -4.47713707,  -2.44819125, ...,   2.43019856,
            1.01633669,   3.3643

## Policy Learning

We now train a counterfactually fair policy using the preprocessor `sp` and the set of trajectory data that is not used to train `sp`. We begin by initializing an `FQI` agent with `sp` as its internal preprocessor.

In [7]:
agent = FQI(num_actions=2, model_type='nn', preprocessor=sp)

We now perform training. Since we set `preprocess=True` in `train()`, `agent` will use its internal preprocessor (i.e. `sp`) to automatically preprocess the input training trajectory before using the trajectory for policy learning. Therefore, we can directly pass in the unpreprocessed `states_prepro` and `rewards_prepro`. 

In [8]:
agent.train(zs=zs_prepro, 
            xs=states_prepro, 
            actions=actions_prepro, 
            rewards=rewards_prepro, 
            max_iter=100, 
            preprocess=True)

100%|██████████| 100/100 [01:14<00:00,  1.35it/s]


## Decision-making Using the Trained Policy

Now we have trained a counterfactually fair policy. We can use its `act()` method to make decisions. At actual deployment, the single-step trajectory passed into `act()` is often real environment data generated at the time. However, in this part, we reuse the training trajectory data for demonstration purposes.

We first take actions for the initial time step (i.e. $a_0$). Again, we only pass in unpreprocessed states and rewards because they will be automatically preprocessed inside `act()` before being used for decision-making. We set `xtm1` and `atm1` to `None`. This tells `agent` that we are taking the initial action and that the buffer of the internal preprocessor should be reset.

In [9]:
a0 = agent.act(z=zs, 
               xt=states[:, 0], 
               xtm1=None, 
               atm1=None, 
               preprocess=True)
a0

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

We now take the second action. We still pass in unpreprocessed states and rewards, and we specify non-`None` `xtm1` and `atm1`.

In [10]:
a1 = agent.act(z=zs, 
               xt=states[:, 1], 
               xtm1=states[:, 0], 
               atm1=a0, 
               preprocess=True)
a1

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

## Alternative: Using All Individuals for Policy Learning

Sometimes, the number of individuals in the trajectory is small. In this case, if we only use a subset of individuals for policy learning, the amount of training trajectory data might be too small to be useful. Fortunately, when we have a relatively large number of `cross_folds`, we can directly preprocess all individuals using the `train_preprocessor()` function, and the preprocessed trajectory can be used for policy learning.

When `cross_folds=K` where `K` is greater than 1, `train_preprocessor()` will internally divide the training data into `K` folds. For each $i=1,\dots,k$, it trains a model based on all the folds other than the $i$-th one, which is then used to preprocess data in the $i$-th fold. This results in `K` folds of preprocessed data, each of which is processed using a model that is trained on the other folds. These `K` folds of data are then combined and returned by `train_preprocessor()`. This method allows us to preprocess all individuals in the trajectory while reducing overfitting.

To use this functionality, we first initialize a `SequentialPreprocessor` with `cross_folds` greater than 1. We use `cross_folds=5` here.

In [11]:
sp_cf5 = SequentialPreprocessor(z_space=[[0, 0], [0, 1], [1, 0], [1, 1]], 
                                num_actions=2, 
                                cross_folds=5, 
                                mode='single', 
                                reg_model='nn')

We now simultaneously train the preprocessor and preprocess all individuals in the trajectory using the precedure described above.

In [12]:
states_tilde_cf5, rewards_tilde_cf5 = sp_cf5.train_preprocessor(zs=zs, 
                                                                xs=states, 
                                                                actions=actions, 
                                                                rewards=rewards)

100%|██████████| 1000/1000 [00:47<00:00, 20.92it/s]


We now train a policy using `FQI` and the preprocessed data. Note that we set `preprocess=False` during training because the training data `state_tilde` and `rewards_tilde` are already preprocessed.

In [13]:
agent_cf5 = FQI(num_actions=2, model_type='nn', preprocessor=sp)
agent_cf5.train(zs=zs, 
                xs=states_tilde_cf5, 
                actions=actions, 
                rewards=rewards_tilde_cf5, 
                max_iter=100, 
                preprocess=False)

100%|██████████| 100/100 [01:46<00:00,  1.07s/it]


We now take the initial action using `agent_cf5`.

In [14]:
a0 = agent.act(z=zs, 
               xt=states[:, 0], 
               xtm1=None, 
               atm1=None, 
               preprocess=True)
a0

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

We now take the second action using `agent_cf5`.

In [15]:
a1 = agent.act(z=zs, 
               xt=states[:, 1], 
               xtm1=states[:, 0], 
               atm1=a0, 
               preprocess=True)
a1

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,